In [ ]:
import redis
import io
import os
import json
from redis.commands.search.query import Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.field import TextField, TagField, NumericField, VectorField
import numpy as np
from numpy.linalg import norm
from PIL import Image
from deepface import DeepFace

REDIS_URL=os.getenv('REDIS_URL', "redis://localhost:6379")
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "Dlib", "ArcFace"]
EMBEDDING_MODEL=models[0]

In [ ]:
# Get a Redis connection
r = redis.Redis.from_url(REDIS_URL, decode_responses=True)
print(r.ping())

In [ ]:
def store_models():
    for person in range(1, 41):
        person = "s" + str(person)
        for face in range(1, 6):
            facepath = '../../data/orl/' + person + "/" + str(face) + '.bmp'
            print ("Training face: " + facepath)
            vec = DeepFace.represent(facepath, model_name=EMBEDDING_MODEL, enforce_detection=False)[0]['embedding']
            embedding = np.array(vec, dtype=np.float32).astype(np.float32).tobytes()
            face_data_values ={ 'person_id':person,
                                'person_path':facepath,
                                'embedding':embedding}
            r.hset('face:'+person+':'+str(face),mapping=face_data_values)

In [ ]:
def create_index():
    indexes = r.execute_command("FT._LIST")
    if "face_idx" not in indexes:
        index_def = IndexDefinition(prefix=["face:"], index_type=IndexType.HASH)
        schema = (VectorField("embedding", "HNSW", {"TYPE": "FLOAT32", "DIM": 2622, "DISTANCE_METRIC": "COSINE"}, as_name="embedding"))
        r.ft('face_idx').create_index(schema, definition=index_def)
        print("The index has been created")
    else:
        print("The index exists")

In [ ]:
def find_face(facepath):
    vec = DeepFace.represent(facepath, model_name=EMBEDDING_MODEL, enforce_detection=False)[0]['embedding']
    embedding = np.array(vec, dtype=np.float32).astype(np.float32).tobytes()

    q = Query("*=>[KNN 1 @embedding $vec AS score]").return_field("score").dialect(2)
    res = r.ft("face_idx").search(q, query_params={"vec": embedding})

    for face in res.docs:
        print(face.id.split(":")[1])
        return face.id.split(":")[1]


def test():
    success = 0
    for person in range(1, 41):
        person = "s" + str(person)
        for face in range(6, 11):
            facepath = '../../data/orl/' + person + "/" + str(face) + '.bmp'
            print ("Testing face: " + facepath)
            found = find_face(facepath)
            if (person == found):
                success = success +1

    print(success/200*100)

In [ ]:
store_models()

In [ ]:
create_index()

In [ ]:
test()